# Preview Support Tickets

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import streamlit as st
import altair as alt
session = get_active_session()

df_support_tickets = session.table('support_tickets')
df_support_tickets

# Automatic ticket categorization using LLM

In [ ]:
prompt = """You are an agent that helps organize requests that come to our support team. 

The request category is the reason why the customer reached out. These are the possible types of request categories:

Roaming fees
Slow data speed
Lost phone
Add new line
Closing account

Try doing it for this request and return only the request category only.
"""

## Use mistral-large

In [ ]:
mistral_large_response_sql = f""" select ticket_id, request, 
trim(snowflake.cortex.complete('mistral-large',concat('{prompt}',request)),'\n') as mistral_large_response
from support_tickets"""

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response

## Compare LLMs

In [ ]:
mistral_7b_response_sql = f""" select ticket_id,
trim(snowflake.cortex.complete('mistral-7b',concat('{prompt}',request)),'\n') as mistral_7b_response
from support_tickets"""

df_mistral_7b_response = session.sql(mistral_7b_response_sql)

df_llms = df_mistral_large_response.join(df_mistral_7b_response,'ticket_id')
df_llms

# Generate dataset to fine-tune mistral-7b

In [ ]:
df_fine_tune = df_mistral_large_response.with_column("prompt", F.concat(F.lit(prompt),F.lit(" "),F.col("request"))).select("ticket_id","prompt","mistral_large_response")
df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune')

# Fine-tune mistral-7b

## Split data into training and evaluation

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune").random_split(weights=[0.8, 0.2], seed=42)
train_df.write.mode('overwrite').save_as_table('support_tickets_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_eval')

In [ ]:
session.table('support_tickets_train').show(1)

In [ ]:
session.table('support_tickets_eval').show(1)

## *Fine-tune mistral-7b model using Cortex*

In [ ]:
select snowflake.cortex.finetune('CREATE', 'CORTEX_FINETUNING_DB.PUBLIC.SUPPORT_TICKETS_FINETUNED_MISTRAL_7B', 'mistral-7b', 'SELECT prompt, mistral_large_response as completion from CORTEX_FINETUNING_DB.PUBLIC.support_tickets_train', 'SELECT prompt, mistral_large_response as completion from CORTEX_FINETUNING_DB.PUBLIC.support_tickets_eval');

### See status of the fine tuning job

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'CortexFineTuningWorkflow_f4016e33-92ce-45d3-918a-19115c398f10');

# Inference using fine-tuned model

In [ ]:
fine_tuned_model_name = 'SUPPORT_TICKETS_FINETUNED_MISTRAL_7B'
sql = f"""select ticket_id, request,
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}',request)),'\n') as fine_tuned_mistral_7b_model_response
from support_tickets"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response

# Auto-generate custom emails and text messages

In [ ]:
df = df_fine_tuned_mistral_7b_response.group_by('fine_tuned_mistral_7b_model_response').agg(F.count("*").as_('COUNT'))
st.subheader("Number of requests per category")

chart = alt.Chart(df.to_pandas()).mark_bar().encode(
    y=alt.Y('FINE_TUNED_MISTRAL_7B_MODEL_RESPONSE:N', sort="-x"),
    x=alt.X('COUNT:Q',),
    color=alt.Color('FINE_TUNED_MISTRAL_7B_MODEL_RESPONSE:N', scale=alt.Scale(scheme='category10'), legend=None),
).properties(height=400)

st.altair_chart(chart, use_container_width=True)

## Streamlit application to auto-generate custom emails and text messages

In [ ]:
st.subheader("Auto-generate custom emails or text messages")

with st.container():
    with st.expander("Edit prompt and select LLM", expanded=True):
        entered_prompt = st.text_area('Prompt',"""Please write an email or text promoting a new plan that will save customers total costs. If the customer requested to be contacted by text message, write text message response in less than 25 words, otherwise write email response in maximum 100 words.""")
    
        with st.container():
            left_col,right_col = st.columns(2)
            with left_col:
                selected_category = st.selectbox('Select category',('Roaming fees', 'Closing account', 'Add new line', 'Slow data speed'))
            with right_col:
                selected_llm = st.selectbox('Select LLM',('snowflake-arctic','llama3-8b','mistral-large', 'reka-flash',))

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generate messages ⚡',type="primary")

with st.container():
    if generate_template:
        sql = f"""select s.ticket_id, s.customer_name, concat(IFF(s.contact_preference = 'Email', '📩', '📲'), ' ', s.contact_preference) as contact_preference, snowflake.cortex.complete('{selected_llm}',
        concat('{entered_prompt}','Here is the customer information: Name: ',customer_name,', Contact preference: ', contact_preference))
        as llm_response from support_tickets as s join support_tickets_train as t on s.ticket_id = t.ticket_id
        where t.mistral_large_response = '{selected_category}' limit 10"""

        with st.status("In progress...") as status:
            df_llm_response = session.sql(sql).to_pandas()
            st.subheader("LLM-generated emails and text messages")
            for row in df_llm_response.itertuples():
                status.caption(f"Ticket ID: `{row.TICKET_ID}`")
                status.caption(f"To: {row.CUSTOMER_NAME}")
                status.caption(f"Contact through: {row.CONTACT_PREFERENCE}")
                status.markdown(row.LLM_RESPONSE.replace("--", ""))
                status.divider()
            status.update(label="Done!", state="complete", expanded=True)